In [1]:
import os
import time
import pandas as pd
import re

In [2]:
# 功能函数
def save2Excel(outputFile, sheetDict):
    currTime = time.time()
    with pd.ExcelWriter(outputFile) as writer:
        for sheetName, resDF in sheetDict.items():
            resDF.to_excel(writer, sheet_name=sheetName, index=False, header=True, encoding="utf-8")
    print(">>报表已保存至：{}".format(outputFile))

In [3]:
def medicineStat(series):
    '''自定义统计各种药的次数'''
    cnt_dict = {} 
    pattern = r'各?\d*g'
    try:
        for row in series:
            medicine_list = row.split(',')
            for medicine in medicine_list:
                medicine = medicine.strip()
                medicine = re.sub(pattern, '', medicine).strip()  # 格式化
                if medicine not in cnt_dict:
                    cnt_dict[medicine] = 1
                else:
                    cnt_dict[medicine] += 1
        df = pd.DataFrame.from_dict(cnt_dict, orient='index').reset_index().rename(columns ={'index':'药物',0:'药物频次'})

        return df
    except Exception as e:
        raise Exception("方法出错，number={},请检查{}".format(row, e))

def stat_further(df,target_df,target_name,medicine_topn):
    target_df[['高频药味数','高频药频次']+medicine_topn] = 0
    pattern = r'各?\d*g'
    for row_num1 in range(target_df.shape[0]):
        df_rows = df[df[target_name] == target_df.loc[row_num1,target_name]]
        for row_num2 in range(df_rows.shape[0]):  # 统计该证型/处方下，频次最高的前n味药的出现次数
            medicine_list = df_rows.reset_index(drop=True).loc[row_num2,'药物'].split(',')
            for medicine in medicine_list:
                medicine = medicine.strip()
                medicine = re.sub(pattern, '', medicine).strip()  # 格式化
                if medicine in medicine_topn:
                    target_df.loc[row_num1,medicine] += 1  # 累计高频药的频次
        # 计算该证型/处方下，高频药的总味数
        target_df.loc[row_num1,'高频药味数'] = target_df.loc[row_num1,medicine_topn][target_df.loc[row_num1,medicine_topn]>0].count()  
        # 计算该证型/处方下，高频药的总频次
        target_df.loc[row_num1,'高频药频次'] = target_df.loc[row_num1,medicine_topn].sum()  
    
def stat_init(inputFile, topn):
    df = pd.read_excel(inputFile)[['证型', '处方','药物']]
    
    syndrome_type_stat_df = df['证型'].value_counts().to_frame()         # 统计各证型出现频次
    syndrome_type_stat_df = syndrome_type_stat_df.reset_index().rename(columns ={'index':'证型','证型':'证型频次'})
    prescription_stat_df = df['处方'].value_counts().to_frame()          # 统计各处方出现频次
    prescription_stat_df = prescription_stat_df.reset_index().rename(columns ={'index':'处方','处方':'处方频次'})
    medicine_stat_df = medicineStat(df['药物'])                          # 统计各药物出现频次
    medicine_stat_df.sort_values(by=['药物频次'], ascending=[False], inplace=True)  # 按照频次降序排序
    medicine_stat_df = medicine_stat_df.reset_index(drop=True)
   
    medicine_topn = list(medicine_stat_df.loc[:topn-1,'药物'])   # 筛选出现频次最高的前n味药（高频药）
    stat_further(df,syndrome_type_stat_df,'证型',medicine_topn)  # 统计该证型下的高频药
    stat_further(df,prescription_stat_df,'处方',medicine_topn)   # 统计该处方下的高频药
        
    return syndrome_type_stat_df,prescription_stat_df,medicine_stat_df

In [4]:
# 参数
inputFile = r'C:\Users\sunjr\Desktop\宝贝\原发性失眠数据挖掘\原发性失眠.xlsx'
outputFile = r'C:\Users\sunjr\Desktop\宝贝\原发性失眠数据挖掘\result.xlsx'
topn = 10  # 出现频次最高的前n味药

In [5]:
# main
syndrome_type_stat_df,prescription_stat_df,medicine_stat_df = stat_init(inputFile, topn)
sheetDict = {'各证型频次统计': syndrome_type_stat_df,'各处方频次统计':prescription_stat_df,'各药物频次统计':medicine_stat_df}
save2Excel(outputFile, sheetDict)

>>报表已保存至：C:\Users\sunjr\Desktop\宝贝\原发性失眠数据挖掘\result.xlsx
